In [12]:
from collections import Counter
import math
import random

In [13]:
# Define each object as a class
class Object:
    def __init__(self, category, values):
        self.category = category
        self.values = values

In [14]:
# Define each node in decision tree as a class
class Node:
    def __init__(self, name):
        self.name = name
        self.childDictionary = {}

In [15]:
# Define entropy values
def entropy(objectList):
    entropy = 0
    for category in yLabels:
        count = 0
        for x in objectList:
            if x.category == category:
                count += 1
        frequency = count/len(objectList) # Count the frequency of categories
        if frequency != 0:
            entropy -= (frequency * math.log(frequency,2)) # Compute Entropy
    return entropy

In [16]:
# Define information gain
def gain(S,A):
    gain = entropy(S) # Compute entropy before splitting
    setOfValues = set([x.values.get(A) for x in S])
    for value in setOfValues:
        l = []
        for x in S:
            if x.values.get(A) == value:
                l.append(x)
        gain -= (len(l)/len(S) * entropy(l)) # Compute information gain after splitting
                    
    return gain

In [17]:
# Create Decision tree based on information gain
def createDecisionTree(objectList, attributeList):
    objectCategory = []
    for x in objectList:
        objectCategory.append(x.category)
    categoryFrequency = Counter(objectCategory)   # Count frequency of categories
    
    if len(categoryFrequency) == 1 or len(attributeList) == 0:
        return Node(categoryFrequency.most_common(1)[0][0])
    
    greatestGain = 0
    greatestA = None
    for attribute in attributeList: # Calculate information gain for all attributes
        g = gain(objectList, attribute)
        if g >= greatestGain:
            greatestGain = g
            greatestA = attribute   # Select the attribute with maximum information gain
    newNode = Node(greatestA)    # Create new nodes based on the attributes with maximum information gain
    setOfValues = set([x.values.get(greatestA) for x in objectList])
    attributeList.remove(greatestA)  # remove used attribute from the dataset
    
    for value in setOfValues:
        newObjectList = []
        for x in objectList:
            if x.values.get(greatestA) == value:
                newObjectList.append(x)    # split the dataset into subset
                
        if len(newObjectList) == 0:
            newNode.name = categoryFrequency.most_common(1)[0][0]
            return newNode   # If all objects are leaf nodes,stop the ID3
        
        newNode.childDictionary[value] = createDecisionTree(newObjectList, attributeList) # group all nodes to create the decision tree
        
    return newNode

In [18]:
# Calculate accuracy from testing data
def accuracy(objectList, root):
    count = 0
    for x in objectList:
       if testNode(x, root) == True:
            count +=1                        # Count the correct prediction
    accuracy = count/len(objectList) * 100   # Compute prediction accuracy
    accuracy = round(accuracy,4)
    
    return accuracy

In [19]:
# Check category in each node
def testNode(obj, node):
    if obj.values[node.name] not in node.childDictionary:
        return False
    node = node.childDictionary[obj.values[node.name]]
    if len(node.childDictionary) == 0:
        if node.name == obj.category:
            return True
        else:
            return False
    return testNode(obj, node)

In [20]:
# Read in a file, attributes, objects, headers
def readFile(filename):
    objects, yLables = [],[]
    inputFile = open(filename, "r")
    attributes = inputFile.readline().strip().split(",") # Read in attributes
    
    for line in inputFile:
        line = line.strip().split(",")
        objects.append(line)     # gather all objects in the file
    
    random.shuffle(objects)
    yLabels = set([x[0] for x in objects])  # Read in y labels
    return attributes, objects, yLabels

In [21]:
# Split the whole dataset into training set and testing set based on train_test_ratio
def TrainTestSplit(objects,TrainTestRatio):
    if TrainTestRatio > 0.9999 or TrainTestRatio < 0.0001:
        print("Invalid Ratio")
        return False
    
    numberOfTraining = int(len(objects) * (TrainTestRatio))
    trainingData,testingData = [],[]
    count = 0
    for obj in objects:
        values = {}
        for num in range(1, len(attributes)):
            values[attributes[num]] = obj[num]            
        if count < numberOfTraining:
            trainingData.append(Object(obj[0], values))   # collect objects in training set
        else:
            testingData.append(Object(obj[0], values))    # collect objects in testing set
        count += 1
    
    return trainingData,testingData

In [22]:
attributes, objects, yLabels = readFile("data.txt")

trainingData,testingData = TrainTestSplit(objects,0.5)

print("The total number of objects: "+ str(len(objects)))
print("The number of training objects: " + str(len(trainingData)))
print("The number of testing objects: " + str(len(testingData)))

decisionTree = createDecisionTree(trainingData, attributes[1:])
acc = accuracy(testingData, decisionTree)

print("The Testing Accuracy is " + str(acc) + "%")

The total number of objects: 24
The number of training objects: 12
The number of testing objects: 12
The Testing Accuracy is 91.6667%
